In [3]:
import re
import time
from json import dumps
import pandas as pd
from sklearn.model_selection import train_test_split 
from kafka import KafkaProducer
from kafka import KafkaConsumer
from json import loads
import pandas as pd
import joblib



ModuleNotFoundError: No module named 'sklearn'

In [ ]:
consumer = KafkaConsumer(
    'test-data',  # Nombre del tema
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    value_deserializer=lambda m: loads(m.decode('utf-8')),
    bootstrap_servers=['localhost:9092']
)

# Lista para almacenar todos los datos recibidos
all_data_received = []
timeout_count = 0

# Consumir mensajes de Kafka
while timeout_count < 3:
    message = consumer.poll(timeout_ms=3000)
    if message:
        for _, records in message.items():
            for record in records:
                # Añadir los valores de los registros a la lista
                all_data_received.append(record.value)
        timeout_count = 0
    else:
        print("Timeout")
        timeout_count += 1

# Cerrar el consumidor de Kafka
consumer.close()

# Crear un DataFrame con los datos recibidos
happy_test = pd.DataFrame(all_data_received)


In [ ]:
my_happiness_model = joblib.load('my_happiness_model.pkl')